# complex model

This notebook creates a model simulating an complex with magnetization using the poligonal prisms from fatiando and save its parameters in an outer file.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os
from scipy import signal

from fatiando import utils
from fatiando.gridder import regular
from fatiando.vis import myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img

C:\Users\bes064\AppData\Local\Continuum\anaconda2\lib\site-packages\fatiando\vis\myv.py:51: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


### Auxiliary functions

In [2]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

C:\Users\bes064\AppData\Local\Continuum\anaconda2\lib\site-packages\fatiando\vis\mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


### Creating the model

In [3]:
complex_model = dict()

In [16]:
# model parameters
complex_model['M'] = 30 # number of vertices per prism
complex_model['L'] = 22 # number of prisms
complex_model['P'] = complex_model['L']*(complex_model['M']+2) # number of parameters

complex_model['z0'] = 200.    # depth of the top the shallowest prism
complex_model['dz'] = 300.    # thickness of each prism

complex_model['inc'] = 65.  # inclination
complex_model['dec'] = -40.5   # declination
complex_model['intensity'] = 10. # intensity

complex_model['props']={'magnetization': utils.ang2vec(
    complex_model['intensity'], complex_model['inc'], complex_model['dec'])} # physical property: magnetization

#ri = np.linspace(1000., 200., complex_model['M']*complex_model['L']) # radial distance for each vertice
#rand = np.random.uniform(0.8, 1., complex_model['M']*complex_model['L'])
#r = ri*rand

#r = np.random.uniform(low=400., high=700., size=complex_model['M'])
#r = signal.medfilt(r)
r = np.array([ 513.8863825 ,  608.66242761,  608.66242761,  639.21917548,
                554.40019092,  651.54224441,  666.06947764,  666.06947764,
                568.66501881,  568.66501881,  625.57161505,  625.57161505,
                522.11037285,  522.11037285,  542.90744237,  542.90744237,
                405.92270651,  405.92270651,  549.6901502 ,  636.54706803,
                551.61570786,  582.77644177,  582.77644177,  582.77644177,
                464.08301193,  464.08301193,  407.89472325,  407.98706731,
                407.98706731,  407.98706731])
rs = []
#r = np.zeros((complex_model['L'],1),dtype=r.dtype) + r
#r = np.repeat(r[None],complex_model['M'],axis=0)

scale = np.array([1.2, 1.4, 1.6, 1.4, 1.2,
                  1., 1., 0.7, 0.6, 0.55,
                  0.55, 0.5, 0.5, 0.7, 1.6,
                  1.8, 2., 2.4, 2.5, 2.6,
                  2.8, 3.])

for i in range(complex_model['L']):
    rs.append(r*scale[i])

In [40]:
# Cartesian coordinates of the origin of each prism
x0 = np.linspace(-2500., 1500., complex_model['L'])
y0 = np.sqrt(2000.**2. + x0**2.) - 2500.

l = []   # list of parameters of the prisms

for i, (r, x, y) in enumerate(zip(rs, x0, y0)):
    l.append([r, x, y,
             complex_model['z0'] + complex_model['dz']*i,
             complex_model['z0'] + complex_model['dz']*(i + 1),
              complex_model['props']])

#for i in range(complex_model['L']):
#    l.append([r*(-150.*i*i/11. + 3150.*i/11. + 500.), x0[i], y0[i], complex_model['z0'] + complex_model['dz']*i,\
#          complex_model['z0'] + complex_model['dz']*(i + 1), complex_model['props']])

complex_model['prisms'] = mfun.pol2cart(l, complex_model['M'], complex_model['L'])
complex_model['param_vec'] = mfun.param_vec(l, complex_model['M'], complex_model['L'])

### Model ploting

In [41]:
bounds = [-3500., 3500., -3500., 3500., -100., 7000.]

red, green, blue = (1, 0, 0), (0, 1, 0), (0, 0, 1)
white, black = (1, 1, 1), (0, 0, 0),

# Show the model
scene = myv.figure()
myv.polyprisms(complex_model['prisms'], 'magnetization', opacity=1., color=blue)

ax = myv.axes(myv.outline(bounds), ranges=[i * 0.001 for i in bounds])
ax.axes.x_label, ax.axes.y_label, ax.axes.z_label = 'x (km)', 'y (km)', 'z (km)'
ax.axes.font_factor = 1.65
scene.scene.camera.position = [-0.089204656936016313, -0.066107174155361687, -0.04083578885723696]
scene.scene.camera.focal_point = [1.1820671636724554e-16, 1.3134079596360617e-17, -1.9701119394540926e-17]
scene.scene.camera.view_angle = 30.0
scene.scene.camera.view_up = [0.24683390337942718, 0.24593292331329802, -0.93733132955911114]
scene.scene.camera.clipping_range = [0.057117737884348194, 0.1956040854922288]
scene.scene.camera.compute_view_plane_normal()

myv.wall_bottom(ax.axes.bounds, opacity=0.2)
myv.wall_north(ax.axes.bounds)

#myv.savefig('../synthetic_data/complex/complex_true.png')
myv.show()
#img(filename='../synthetic_data/complex/complex_true.png')

### Saving in an outer file